## Deep Learning Coding Project 2: Image Classification

Before we start, please put your **Chinese** name and student ID in following format:

Name, 0000000000 // e.g.) 傅炜, 2021123123

袁樱, 2020010930

## Introduction

We will use Python 3, [NumPy](https://numpy.org/), and [PyTorch](https://pytorch.org/) for this coding project. The example code has been tested under the latest stable release version.

### Task

In this notebook, you need to train a model to classify images. Given an image, you need to distinguish its category,
e.g., whether it is a horse or an automobile. There are total 10 classes:
airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. We
release 40,000 images for training, 10,000 images for validation. Each image has
a shape of (3, 128, 128). We will evaluate your model in 10,000 images on the test set.

Download the dataset from [here](https://cloud.tsinghua.edu.cn/d/00e0704738e04d32978b/) and organize them into a folder named "cifar_10_4x".

<!-- Images can be classified as "No Finding" or **one or more types**. In the basic task, given an image, you only need to tell whether the X-ray indicates "Infiltration". In the bonus task, you need to tell whether *each* of the diseases exists.

Images are taken from the [ChestX-ray14 dataset](https://www.kaggle.com/nih-chest-xrays/data) and downsampled to (256, 256). We release 44872 gray scale images for training and validation. We will evaluate your model on 10285 images in the test set. The dataset is available [here](https://cloud.tsinghua.edu.cn/d/16d06a89c5b4459db703/) and organized as follows: `train` directory includes all images for training and validation, and each line of `train.txt` records the labels separated by "|". -->

### Coding

We provide a code template. You can add new cells and modify our example to train your own model. To run this code, you should:

+ implement your model (named `Net`) in `model.py`.
+ implement your training loop in this notebook

Your final submitted model should not be larger than **20M**. **Using any pretrained model is NOT permitted**.
Besides, before you submit your result, **make sure you can test your model using our evaluation cell.** Name your best model "cifar10_4x_best.pth".

### Report & Submission

Your report should include:

1. the details of your model
2. all the hyper-parameters
3. all the tricks or training techniques you use
4. the training curve of your submitted model.

Reporting additional ablation studies and how you improve your model are also encouraged.

You should submit:

+ all codes
+ the model checkpoint (only "cifar10_4x_best.pth")
+ your report (a separate "pdf")

to web learning. We will use the evaluation code in this notebook to evaluate your model on the test set.

### Grading

We will grade this coding project based on the performance of your model (70%) and your report (30%). Regarding the evaluation metric of your model, assume your test accuracy is $X$, then your score is

$\frac{min(X,H)−0.6}{H−0.6}×7$

where $H$ is accuracy of the model trained by TAs and $H=0.9$, i.e., you will get the full score if your test accuracy is above 90%.

**Bonus**: The best submission with the highest testing accuracy will get 1 bonus point for the final course grade.

**Avoid plagiarism! Any student who violates academic integrity will be seriously dealt with and receive an F for the course.**

## Code Template

We have masked the the training loop in this notebook for you to complete. You should also overwrite "model.py" and implement your own model.

In [ ]:
%load_ext autoreload
%autoreload 2

### Setup Code

If you use Colab in this coding project, please uncomment the code, fill the `GOOGLE_DRIVE_PATH_AFTER_MYDRIVE` and run the following cells to mount your Google drive. Then, the notebook can find the required file. If you run the notebook locally, you can skip the following cells.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os

# # TODO: Fill in the Google Drive path where you uploaded the assignment
# # Example: If you create a 2022SP folder and put all the files under CP1 folder, then '2022SP/CP1'
# # GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2022SP/CP1'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = None 
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))

In [ ]:
# import sys
# sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
from dataset import CIFAR10_4x
from evaluation import evaluation

from model import Net  # this should be implemented by yourself

### Enjoy Your Coding Time!

In [ ]:
import math
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from PIL import Image
import argparse

def set_seed(seed):
    seed = int(seed)
    if seed < 0 or seed > (2**32 - 1):
        raise ValueError("Seed must be between 0 and 2**32 - 1")
    else:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

parser = argparse.ArgumentParser(description="DL Coding2")
parser.add_argument('--model', default='resnet18')
args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
set_seed(16)

In [ ]:
data_root_dir = '.'

transform = transforms.Compose([
        # transforms.Resize((16, 16)),
        transforms.RandomChoice([
            transforms.RandomResizedCrop(128),
            transforms.RandomAffine(15),
            transforms.RandomHorizontalFlip(p=0.4)
        ]),
        transforms.RandomGrayscale(p=0.1),
        # transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([125 / 255, 124 / 255, 115 / 255],
                            [60 / 255, 59 / 255, 64 / 255])
    ])

trainset = CIFAR10_4x(root=data_root_dir,
                      split="train", transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)

validset = CIFAR10_4x(root=data_root_dir,
                      split='valid', transform=transform)
validloader = torch.utils.data.DataLoader(
    validset, batch_size=128, shuffle=False, num_workers=8)

if args.model == 'effnet':
        from models import effnet
        net = effnet.EfficientNet.from_name('efficientnet-b0')

print("number of trained parameters: %d" % (
    sum([param.nelement() for param in net.parameters() if param.requires_grad])))
print("number of total parameters: %d" %
      (sum([param.nelement() for param in net.parameters()])))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3) # weight_decay=5e-5
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=(75, 90), gamma=0.4, last_epoch=-1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500, eta_min=0, last_epoch=-1)

net.to(device)

In [2]:
model_dir = '.'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
torch.save(net, os.path.join(model_dir, 'cifar10_4x_0.pth'))

# check the model size
os.system(' '.join(['du', '-h', os.path.join(model_dir, 'cifar10_4x_best.pth')]))


 20M	./cifar10_4x_best.pth


0

In [ ]:
import json
def train():
    ##############################################################################
    #                  TODO: You need to complete the code here                  #
    ##############################################################################
    # YOUR CODE HERE
    loss_tracking = []
    acc = []
    print("Begin training!")
    for epoch in range(500):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
                loss_tracking.append(running_loss / 200)
                with open('loss1.json', 'a') as f:
                    json.dump(str(running_loss / 200), f)
                running_loss = 0.0
        if epoch % 10 == 9:
            torch.save(net, os.path.join(model_dir, (str)(epoch) + '.pth'))
            acc_valid = evaluation(net, validloader, device)
            acc_train = evaluation(net, trainloader, device)
            with open('acc_valid9.txt', 'a') as f:
                json.dump(str(acc_valid), f)
            with open('acc_train9.txt', 'a') as f:
                json.dump(str(acc_train), f)

        scheduler.step()

    print("Finished Training")
    ##############################################################################
    #                              END OF YOUR CODE                              #
    ##############################################################################
train()

## Evaluation

Before submission, please run the following cell to make sure your model can be correctly graded.

In [1]:
!python evaluation.py

number of trained parameters: 5288548
number of total parameters: 5288548
can't load test set because [Errno 2] No such file or directory: 'cifar_10_4x/test', load valid set now
 20M	cifar10_4x_best.pth
Accuracy of the network on the valid images: 94 %
